# Perform linear model per CellProfiler feature on concatenated normalized data from plates 5, 3 prime, and 3

We will include 3 co-variates:

1. Cell count per well contribution
2. Plate contribution
3. Genotype contribution (WT versus Null)

In [1]:
import pathlib

import pandas as pd
from pycytominer import feature_select
from pycytominer.cyto_utils import infer_cp_features
from sklearn.linear_model import LinearRegression

In [2]:
# Define inputs and outputs
data_dir = pathlib.Path("../../../3.processing_features/data/single_cell_profiles/")
cp_files = [
    pathlib.Path(data_dir, f"Plate_{plate}_sc_normalized.parquet")
    for plate in ["5", "3", "3_prime"]
]

# Output directory for LM coeffs
output_dir = pathlib.Path("./results")
output_dir.mkdir(exist_ok=True)

# Name of output file with LM coeffs
output_cp_file = pathlib.Path(
    output_dir, "linear_model_cp_features_concat_plate5_plate3_plate3prime.tsv"
)

# Filter and load the specified files
df_list = []

for plate in cp_files:
    cp_file = pathlib.Path(plate)

    if cp_file.exists():
        # Load the parquet file into a DataFrame without HET rows
        df = pd.read_parquet(cp_file, filters=[('Metadata_genotype', '!=', 'HET')])

        # Update Metadata_Plate only for Plate_3_prime since during CellProfiler analysis, the metadata only caught Plate_3
        if plate.stem.replace("_sc_normalized", "") == "Plate_3_prime":
            df["Metadata_Plate"] = "Plate_3_prime"

        df_list.append(df)

# Concatenate the DataFrames
concat_df = pd.concat(df_list, ignore_index=True)

# Make sure there are no NaNs
concat_df = feature_select(concat_df, operation="drop_na_columns", na_cutoff=0)

# Define CellProfiler features
cp_features = infer_cp_features(concat_df)

print(f"We are testing {len(cp_features)} CellProfiler features")
print(concat_df.shape)
concat_df.head()

We are testing 2271 CellProfiler features
(22585, 2289)


,Metadata_WellRow,Metadata_WellCol,Metadata_Well,Metadata_Site,Metadata_number_of_singlecells,Metadata_gene_name,Metadata_genotype,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,...,Nuclei_Texture_Variance_DAPI_3_03_256,Nuclei_Texture_Variance_GFP_3_00_256,Nuclei_Texture_Variance_GFP_3_01_256,Nuclei_Texture_Variance_GFP_3_02_256,Nuclei_Texture_Variance_GFP_3_03_256,Nuclei_Texture_Variance_RFP_3_00_256,Nuclei_Texture_Variance_RFP_3_01_256,Nuclei_Texture_Variance_RFP_3_02_256,Nuclei_Texture_Variance_RFP_3_03_256,Metadata_seed_density
0,B,1,B1,10,79,NF1,WT,870.435339,133.774194,863.193505,...,-0.585393,-0.523726,-0.514327,-0.526490,-0.522161,0.214989,0.273449,0.214783,0.183755,NaN
1,B,1,B1,11,79,NF1,WT,827.549320,342.283025,810.793536,...,-0.748387,-0.465471,-0.445058,-0.464277,-0.450122,-0.258725,-0.260480,-0.263612,-0.256463,NaN
2,B,1,B1,11,79,NF1,WT,427.937346,356.977306,406.334199,...,0.221558,0.444134,0.547131,0.644409,0.433607,-0.385501,-0.384362,-0.374862,-0.380044,NaN
3,B,1,B1,11,79,NF1,WT,272.036245,389.802436,282.897144,...,-0.219022,-0.230114,-0.254531,-0.234149,-0.138286,-0.328111,-0.317422,-0.331309,-0.330312,NaN
4,B,1,B1,11,79,NF1,WT,944.416824,736.917498,963.654663,...,-0.217203,0.208971,0.193882,0.140452,0.136724,-0.498345,-0.493748,-0.496570,-0.494255,NaN


## Set up binary framework for WT versus Null and plate to plate comparison

In [3]:
# Setup linear modeling framework
variables = ["Metadata_number_of_singlecells"]
X = concat_df.loc[:, variables]

# Add binary matrix of categorical genotypes
genotype_x = pd.get_dummies(data=concat_df.Metadata_genotype)

# Add binary matrix of categorical plates
plate_x = pd.get_dummies(data=concat_df.Metadata_Plate)

X = pd.concat([X, genotype_x, plate_x], axis=1)

print(X.shape)
X.head()

(22585, 6)


,Metadata_number_of_singlecells,Null,WT,Plate_3,Plate_3_prime,Plate_5
0,79,0,1,0,0,1
1,79,0,1,0,0,1
2,79,0,1,0,0,1
3,79,0,1,0,0,1
4,79,0,1,0,0,1


In [4]:
# Fit linear model for each feature
lm_results = []
for cp_feature in cp_features:
    # Subset CP data to each individual feature (univariate test)
    cp_subset_df = concat_df.loc[:, cp_feature]

    # Fit linear model
    lm = LinearRegression(fit_intercept=True)
    lm_result = lm.fit(X=X, y=cp_subset_df)

    # Extract Beta coefficients
    # (contribution of feature to X covariates)
    coef = lm_result.coef_

    # Estimate fit (R^2)
    r2_score = lm.score(X=X, y=cp_subset_df)

    # Add results to a growing list
    lm_results.append([cp_feature, r2_score] + list(coef))

In [5]:
# Convert results to a pandas DataFrame
lm_results = pd.DataFrame(
    lm_results,
    columns=[
        "feature",
        "r2_score",
        "cell_count_coef",
        "Null_coef",
        "WT_coef",
        "Plate_3_coef",
        "Plate_3_prime_coef",
        "Plate_5_coef",
    ],
)

# Output file
lm_results.to_csv(output_cp_file, sep="\t", index=False)

print(lm_results.shape)
lm_results.head()

(2271, 8)


,feature,r2_score,cell_count_coef,Null_coef,WT_coef,Plate_3_coef,Plate_3_prime_coef,Plate_5_coef
0,Cytoplasm_AreaShape_Area,0.076596,-0.001731,-0.036435,0.036435,0.350194,-0.068791,-0.281403
1,Cytoplasm_AreaShape_BoundingBoxArea,0.076316,-0.001628,-0.052330,0.052330,0.320520,-0.074655,-0.245865
2,Cytoplasm_AreaShape_BoundingBoxMaximum_X,0.000805,-0.000147,-0.014859,0.014859,0.034820,-0.014360,-0.020460
3,Cytoplasm_AreaShape_BoundingBoxMaximum_Y,0.002924,-0.000333,-0.003757,0.003757,0.075917,-0.016994,-0.058923
4,Cytoplasm_AreaShape_BoundingBoxMinimum_X,0.000846,0.000185,-0.003690,0.003690,-0.033054,0.001485,0.031569
